## Novel model approach

This notebook serves as the source code for all the model testing and training (along with hyperparam grid search) before the development/submission of the final best model. This model approach tries a variation on the transformer architecture, with different heads, as detailed in the report.

In [3]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer
from sklearn.metrics import f1_score, average_precision_score
import itertools
from tqdm import tqdm

In [4]:
from pcl_tf.dataset_manager import DatasetManager as DM
from pcl_tf.collation import collate_fn
from pcl_tf.tf import warmup_model, get_tokenizer, _ENCODER_CACHE

In [5]:
NUM_LABELS = 7
LOAD_BATCH_SIZE = 64
LOCAL_CACHE_DIR = './models_cache'
MODEL_NAME = "roberta-base"
NUM_WORKERS = 8
PIN_MEMORY = False
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")

Using device: cuda


In [6]:
print('Warming up tokenizer...')
tokenizer = get_tokenizer(MODEL_NAME)

print('Warming up encoder (downloads model if needed)...')

_ = warmup_model(MODEL_NAME, device=None, cache_dir=LOCAL_CACHE_DIR)
print('Model cache warmup completed.')

Warming up tokenizer...


Warming up encoder (downloads model if needed)...


Loading weights:   0%|          | 0/197 [00:00<?, ?it/s]

RobertaModel LOAD REPORT from: roberta-base
Key                             | Status     | 
--------------------------------+------------+-
lm_head.dense.weight            | UNEXPECTED | 
lm_head.layer_norm.weight       | UNEXPECTED | 
lm_head.bias                    | UNEXPECTED | 
lm_head.dense.bias              | UNEXPECTED | 
roberta.embeddings.position_ids | UNEXPECTED | 
lm_head.layer_norm.bias         | UNEXPECTED | 
pooler.dense.weight             | MISSING    | 
pooler.dense.bias               | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


Model cache warmup completed.


In [7]:
train_labels_path = "data/train_semeval_parids-labels.csv"
dev_labels_path = "data/dev_semeval_parids-labels.csv"
texts_path = "data/dontpatronizeme_pcl_cleaned.csv"
test_path = "data/task4_test.tsv"
cats_path = "data/dontpatronizeme_categories.tsv"

In [8]:
texts_df = pd.read_csv(texts_path, low_memory=False)
texts_df["par_id"] = texts_df["par_id"].astype(int)
texts_df = texts_df.set_index("par_id")

In [9]:
training_ds = DM(train_labels_path, texts_df=texts_df)
training_ds.print_stats()

Total samples: 8375
Binary distribution: [7581  794]
Multilabel distribution: [574. 160. 162. 192. 145. 363.  29.]


In [10]:
dev_ds = DM(dev_labels_path, texts_df=texts_df)
dev_ds.print_stats()

Total samples: 2093
Binary distribution: [1894  199]
Multilabel distribution: [142.  36.  62.  38.  52. 106.  11.]


In [11]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)


train_loader = DataLoader(training_ds, batch_size=LOAD_BATCH_SIZE, shuffle=True, 
                          collate_fn=lambda b: collate_fn(tokenizer, b), pin_memory=PIN_MEMORY, num_workers=NUM_WORKERS)

dev_loader = DataLoader(dev_ds, batch_size=LOAD_BATCH_SIZE, shuffle=False, 
                        collate_fn=lambda b: collate_fn(tokenizer, b), pin_memory=PIN_MEMORY, num_workers=NUM_WORKERS)

In [12]:
def evaluate_dev(model, dataloader, device):
    model.eval()
    bin_probs=[]
    bin_labels=[]
    multi_probs=[]
    multi_labels=[]

    with torch.no_grad():
        for b in dataloader:
            input_ids = b["input_ids"].to(device)
            attention_mask = b["attention_mask"].to(device)
            labels = b["labels"].to(device)
            out = model(input_ids=input_ids, attention_mask=attention_mask)
            bin_probs.append(torch.sigmoid(out["logit_bin"]).cpu().numpy())
            multi_probs.append(torch.sigmoid(out["logit_multi"]).cpu().numpy())
            bin_labels.append(labels[:,0].cpu().numpy())
            multi_labels.append(labels[:,1:].cpu().numpy())
            
    bin_probs = np.concatenate(bin_probs); bin_labels = np.concatenate(bin_labels)
    multi_probs = np.concatenate(multi_probs); multi_labels = np.concatenate(multi_labels)
    micro_f1 = f1_score(multi_labels.flatten(), (multi_probs>=0.5).astype(int).flatten(), zero_division=0)
    return {"micro_f1": micro_f1, "bin_ap": average_precision_score(bin_labels, bin_probs)}


In [13]:
from pcl_tf.tf import PCLModel

def train_and_eval(config):
    model = PCLModel(config["model_name"], n_labels=NUM_LABELS, dropout=config["dropout"], device=DEVICE).to(DEVICE)
    optim = torch.optim.AdamW(model.parameters(), lr=config["lr"], weight_decay=config["wd"])
    print("Model and optimizer created")

    print("Starting training...")
    for epoch in range(config["epochs"]):
        model.train()
        running = 0.0
        for batch in train_loader:
            input_ids = batch["input_ids"].to(DEVICE, non_blocking=True)
            attention_mask = batch["attention_mask"].to(DEVICE, non_blocking=True)
            labels = batch["labels"].to(DEVICE, non_blocking=True)
            optim.zero_grad()
            out = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = out["loss"]
            loss.backward()
            optim.step()
            running += loss.item()
        # optionally print per-epoch
        print(f"Epoch {epoch+1}, Loss: {running/len(train_loader)}")
    metrics = evaluate_dev(model, dev_loader, DEVICE)
    return metrics

In [14]:
grid = {
    "model_name": ["roberta-base"],
    "lr": [2e-5, 5e-5],
    "wd": [1e-3, 1e-2, 5e-3],
    "max_len": [128, 256],
    "dropout": [0, 0.1],
    "epochs": [6]
}

In [15]:
torch.cuda.empty_cache()

In [16]:
keys, values = zip(*grid.items())
results = []
for combo in tqdm(list(itertools.product(*values)), desc="Grid"):
    cfg = dict(zip(keys, combo))
    try:
        metrics = train_and_eval(cfg)
        results.append({**cfg, **metrics})
        print("CFG:", cfg, "=>", metrics)
    except Exception as e:
        print("Error for cfg", cfg, ":", e)
        results.append({**cfg, "error": str(e)})

res_df = pd.DataFrame(results)
res_df.to_csv("grid_results.csv", index=False)
print("Saved grid_results.csv")
best_idx = res_df["micro_f1"].idxmax() if "micro_f1" in res_df.columns else None
if best_idx is not None:
    print("Best config:\n", res_df.loc[best_idx].to_dict())
else:
    print("No successful runs found.")

Grid:   0%|          | 0/24 [00:00<?, ?it/s]

Model and optimizer created
Starting training...
Epoch 1, Loss: 0.45656906318573554
Epoch 2, Loss: 0.28324812241182984
Epoch 3, Loss: 0.20947086287818792
Epoch 4, Loss: 0.14777101606187473
Epoch 5, Loss: 0.09722598522674036
Epoch 6, Loss: 0.0835050648177853


Grid:   4%|▍         | 1/24 [07:16<2:47:23, 436.67s/it]

CFG: {'model_name': 'roberta-base', 'lr': 2e-05, 'wd': 0.001, 'max_len': 128, 'dropout': 0, 'epochs': 6} => {'micro_f1': 0.273224043715847, 'bin_ap': 0.5950159538533516}
Model and optimizer created
Starting training...
Epoch 1, Loss: 0.4490828883784418
Epoch 2, Loss: 0.29092072905929944
Epoch 3, Loss: 0.22693791476483563
Epoch 4, Loss: 0.15818579318641707
Epoch 5, Loss: 0.1065674459291551
Epoch 6, Loss: 0.07784430829111401


Grid:   8%|▊         | 2/24 [14:33<2:40:12, 436.95s/it]

...

CFG: {'model_name': 'roberta-base', 'lr': 5e-05, 'wd': 0.005, 'max_len': 128, 'dropout': 0.1, 'epochs': 6} => {'micro_f1': 0.3286384976525822, 'bin_ap': 0.5540698397455713}
Model and optimizer created
Starting training...
Epoch 1, Loss: 0.4094110709679036
Epoch 2, Loss: 0.2792855397094297
Epoch 3, Loss: 0.20516147908136134
Epoch 4, Loss: 0.13374803553669506
Epoch 5, Loss: 0.09884872769524577
Epoch 6, Loss: 0.07762129542254309


Grid:  96%|█████████▌| 23/24 [2:40:11<07:14, 434.96s/it]

CFG: {'model_name': 'roberta-base', 'lr': 5e-05, 'wd': 0.005, 'max_len': 256, 'dropout': 0, 'epochs': 6} => {'micro_f1': 0.31347962382445144, 'bin_ap': 0.5089862991043769}
Model and optimizer created
Starting training...
Epoch 1, Loss: 0.4027164536350556
Epoch 2, Loss: 0.29668836835913986
Epoch 3, Loss: 0.21055090225493636
Epoch 4, Loss: 0.15677081836202672
Epoch 5, Loss: 0.10435222879848408
Epoch 6, Loss: 0.08140688053979218


Grid: 100%|██████████| 24/24 [2:47:28<00:00, 418.68s/it]

CFG: {'model_name': 'roberta-base', 'lr': 5e-05, 'wd': 0.005, 'max_len': 256, 'dropout': 0.1, 'epochs': 6} => {'micro_f1': 0.30973451327433627, 'bin_ap': 0.5267868684231509}
Saved grid_results.csv
Best config:
 {'model_name': 'roberta-base', 'lr': 2e-05, 'wd': 0.001, 'max_len': 256, 'dropout': 0.1, 'epochs': 6, 'micro_f1': 0.3930942895086321, 'bin_ap': 0.6005527050861326, 'error': nan}
